# Домашнее задание. Урок 4.

## Практическое задание

Вариант 1. (простой)

- обучить сверточную нейронную сеть в стиле AlexNet (с падением размера ядра свертки и последовательностью блоков свертка-пулинг  (conv-pool)-(conv-pool)-...) на датасете fashion-mnist или cifar10
- оценить рост точности при увеличении ширины сети (больше ядер)
- оценить рост точности при увеличении глубины сети (больше слоев)
- (опционально)сравнить с точностью полносвязной сети для этой выборки

## Особенности сети в стиле AlexNet

1. Последовательное уменьшение размерности ядра свертки
2. Последовательность блокок свертка-пулинг (conv-pool)-(conv-pool)

![](AlexNet.png)

In [ ]:
from __future__ import print_function
#import tensorflow.keras as keras # расскоментируйте эту строку, чтобы начать обучение
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import os

In [1]:
classes = ['самолет', 'автомобиль', 'птица', 'кот', 'олень', 'собака', 'лягушка', 'лошадь', 'корабль', 'грузовик']


## Установка параметров нейросети

In [ ]:
batch_size = 32
num_classes = 10
epochs = 1
data_augmentation = False
num_predictions = 20
# save_dir = os.path.join(os.getcwd(), 'saved_models')
# model_name = 'keras_cifar10_trained_model.h5'


## Разделение тренировочной и тестовой выборки

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'тренировочные примеры')
print(x_test.shape[0], 'тестовые примеры')

## Преобразование матрицы чисел 0-9 в бинарную матрицу чисел 0-1

In [ ]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255



## Конфигурирование нейросети

In [ ]:
def bild_model(koef_core = 1, num_layers = 1):
    model = Sequential()

    # слои нейросети отвественные за свертку и max-pooling
    model.add(Conv2D(32 * koef_core, (3, 3), padding='same', input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))

    for _ in range(num_layers):
        model.add(Conv2D(32 * koef_core, (3, 3)))
        model.add(Activation('relu'))

    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64 * koef_core, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64 * koef_core, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Dropout(0.25))

    # полносвязные слои нейронной сети
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    # компиляция модели
    model.compile(loss='categorical_crossentropy',
                  optimizer='SGD',
                  metrics=['accuracy'])

    return model

In [ ]:
model = bild_model()
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)



In [ ]:
# проверка работы обученной модели
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])